# A Short Introduction to Coq


In this tutorial, we are going to play with [Coq](https://coq.inria.fr/), which is a popular proof assitant based on solid [type theories](https://en.wikipedia.org/wiki/Calculus_of_constructions).

This tutorial contains the following content:

- Basic functional programming in Coq
- Curry-Howard correspondence
- First-order Logic
- Proof by tactics
- The equivalence between LEM and DNE

In the above, LEM refers to the [law of excluded middle](https://en.wikipedia.org/wiki/Law_of_excluded_middle), DNE refers to the [law of double negation](https://en.wikipedia.org/wiki/Double_negation).

After this tutorial, we hope that

- you understand how Curry-Howard corespondence is embodied in Coq
- you can do simple proofs in Coq with and without tactics

## Basic Functional Programming in Coq

The core of Coq is a functional programming language, called Gallina. It offers features like _algebraic data types_, _pattern matching_, _parametric polymorphism_, as commonly supported by functional languages.

We create a playground, so that the names will not clash with definitions from Coq.

In [ ]:
Module FPPlayground.

We may define a type for booleans as follows:

In [ ]:
Inductive bool : Set :=
  | true
  | false.

The type `Set` indicates that the type `bool` is not a proposition, but a "value" type.

With the definition above, we can define the common boolean operations:

In [ ]:
Definition negb (b:bool) : bool :=
  match b with
  | true => false
  | false => true
  end.

Definition andb (b1:bool) (b2:bool) : bool :=
  match b1 with
  | true => b2
  | false => false
  end.

Definition orb (b1:bool) (b2:bool) : bool :=
  match b1 with
  | true => true
  | false => b2
  end.

Natural numbers can be defined as follows:

In [ ]:
Inductive nat : Set :=
  | O
  | S (n : nat).

Now we can define the predecessor function:

In [ ]:
Definition pred (n : nat) : nat :=
  match n with
    | O => O
    | S n' => n'
  end.

Let's now define a function that doubles its argument:

In [ ]:
Definition double (n : nat) : nat :=
  match n with
    | O => O
    | S n' => S (S (double n'))
  end.

Oops! Coq complains that `double` was not found. We need to use the keyword `Fixpoint`:

In [ ]:
Fixpoint double (n : nat) : nat :=
  match n with
    | O => O
    | S n' => S (S (double n'))
  end.

Why the complexity? The reason is that in the above, `double` is a recursive function. You might remember from TAPL that unrestricted general recursion can make any type inhabited. By Curry-Howard correspondance, it means that any proposition can be proved true!

Consequently, recursive functions must terminate in order to be accepted by Coq. Coq uses a simple mechanism to check termination of recursive calls, namely _structural recursion_: the recursive call must take an argument which is _structurally_ smaller.

In [ ]:
Fixpoint plus (n : nat) (m : nat) : nat :=
  match n with
    | O => m
    | S n' => S (plus n' m)
  end.

**Exercise 1**: Factorial

Please implement the `factorial` function given below:

In [ ]:
Fixpoint factorial (n:nat) : nat
  (* := ??? *). Admitted.

In [ ]:
End FPPlayground.

## Curry-Howard correspondence

As we learned from class lectures, [Curry-Howard correspondence](https://en.wikipedia.org/wiki/Curry%E2%80%93Howard_correspondence) plays a critical role in proof assitants that are based on type theories. The key insight is that

- proofs are programs, and
- propositions are types

To show that a proposition is provable, it suffices to show that the corresponding types are inhabited by a program. In this section we will see how this correspondence is embodied in Coq.

**Implication**. The most important correspondence is between the function type `A -> B` and implication `A -> B`. Consequently, the proof of `A -> B` is a function of the type `A -> B`. By the [BHK interpretation](https://en.wikipedia.org/wiki/Brouwer%E2%80%93Heyting%E2%80%93Kolmogorov_interpretation) of intuitionist logic, a function that proves `A -> B` basically transforms the proof of `A` to the proof of `B`.


**Conjunction**. The proposition `A /\ B` is represented by a product type:


```Coq
Inductive and (A B:Prop) : Prop :=
  conj : A -> B -> A /\ B
where "A /\ B" := (and A B) : type_scope.
```

Note that the constructor `conj` has the type `A -> B -> A /\ B`, which can be read as _given a proof of A and a proof of B, then we can construct a proof of A /\ B_.


In the above, the type `Prop` refers to propositions.

**Disjunction**. A disjunction `A \/ B` means either we have a proof of `A` or a proof of `B`, thus it is naturally represented by a sum type:

```Coq
Inductive or (A B:Prop) : Prop :=
  | or_introl : A -> A \/ B
  | or_intror : B -> A \/ B
where "A \/ B" := (or A B) : type_scope.
```

**If and only if**. The proposition `A <-> B` is embodied by the type `(A -> B) /\ (B -> A)`, consequently the proof will be a tuple of functions.

```Coq
Definition iff (A B:Prop) := (A -> B) /\ (B -> A).
Notation "A <-> B" := (iff A B) : type_scope.
```

**False**. Which type corresponds to the proposition `False`? As we can never prove `False`, it should correspond to a type that is not inhabited. This can be done in Coq by defining an inductive type without any constructors, thus it is impossible for a term to inhabit the type.

```Coq
Inductive False : Prop :=.
```

**Negation**. How to represent the proposition `~A` in types? In intuitionistic logic, `~A` is interpreted as `A -> False`, i.e., a proof of `A` will lead to absurdity.

```Coq
Definition not (A:Prop) := A -> False.
```

Consequently, `~~A` is the same as `(A -> False) -> False`.

As an exercise, let's prove the following theorem, which says that for any proposition `P`, we may prove `~~P` from `P`:

In [ ]:
Definition neg_fun_prop: Prop := forall P: Prop, P -> ~~P.

The proof is just a function that has the type `neg_fun_prop`:

In [ ]:
Definition neg_fun_proof := fun (P:Prop) (p: P) (np: ~P) => np p.

Let's check the type to see it's indeed the proof:

In [ ]:
Check neg_fun_proof.

The type is equivalent to `neg_fun_prop`:

In [ ]:
Check neg_fun_proof: neg_fun_prop.

## First-order logic

So far, what we have seen are formulas of propositional logic. You might be wondering, what about first-order logic formulas, e.g. $\forall x \in A.P(x)$ and $\exists x \in A.P(x)$? That leads us to [dependent types](https://en.wikipedia.org/wiki/Dependent_type). Coq is based on a dependent type theory called [calculus of inductive constructions](https://coq.inria.fr/distrib/current/refman/language/cic.html).

The most important type in dependent type theories is the $\Pi$-type, which is of the form $\Pi_{x:A}B(x)$. $\Pi$ types denote _dependent functions_, whose return _type_ depends on the parameter of the function. In Coq, the $\Pi$ type is written as `forall x:A, B`. We illustrate by proving the following theorem:

$$\forall x \in nat. even(double(x))$$


First, we reproduce the definitions of `nat` and `double` below.

In [ ]:
Module FOPlayground.

Inductive nat : Type :=
  | O
  | S (n : nat).

Fixpoint double (n : nat) : nat :=
  match n with
    | O => O
    | S n' => S (S (double n'))
  end.

Next, we need to define the predicate `even`: 

In [ ]:
Inductive even : nat -> Prop :=
  | even0 : even O
  | evenS : forall x:nat, even x -> even (S (S x)).

The definition says that `O` is even, and if `x` is even, then `S S x` is even. Now we can define the proposition formally:

In [ ]:
Definition even_prop: Prop := forall x:nat, even (double x).

For the proof, we will need a clever helper function `even_rec`, which is defined below:

In [ ]:
Fixpoint even_rec(m: nat)(p0: (even (double O)))(pS: forall n:nat, (even (double n)) -> (even (double (S n)))): even (double m) :=
  match m with
    | O => p0
    | S n' => pS n' (even_rec n' p0 pS)
  end.

Note that in the above, the recursion is well-founded, because it is structurally decreasing on `m`. With the helper function, we may write the proof, which is a dependent function:

In [ ]:
Definition even_proof :=
  fun n => even_rec n even0 (fun m evenN => (evenS (double m) evenN)).

We can check the type to see we actually proved the theorem:

In [ ]:
Check even_proof.

This example reveals the essence of _proof by induction_ from the perspective of type theory: it is just structural recursion.

In [ ]:
End FOPlayground.

From the above, it is clear that universal quantification can be encoded as $\Pi$ types. But what about existential quantification? In Coq, existential quantification is encoded as follows:

```Coq
Inductive ex (A:Type) (P:A -> Prop) : Prop :=
  ex_intro : forall x:A, P x -> ex (A:=A) P.
```

Equality (strictly speaking _propositional equality_) is also encoded by an inductive type:

```Coq
Inductive eq (A:Type) (x:A) : A -> Prop :=
    eq_refl : x = x :>A
```

And the type `A -> B` is a special case of $\Pi$ types:

```Coq
Notation "A -> B" := (forall (_ : A), B) : type_scope.
```

You may find more information about the encoding of logic in Coq here: https://coq.inria.fr/stdlib/Coq.Init.Logic.html.

**Exercise 2**: Define an inductive predicate `odd`, and prove that `forall n:nat, odd (S (double n))`.

## Introduction to proofs by tactics

As you have seen in previous examples, doing proofs by writing functions is very different from how proofs are usually done. Fortunately, Coq comes with a language called [_Ltac_,](https://coq.inria.fr/refman/proof-engine/ltac.html), which enables an imperative proving style that is more natural. Ltac also makes it possible to define heuristics to automate some proofs. Ltac only helps to synthesize the proofs, thus even if there are bugs in Ltac, it is not an issue --- all programs or proofs have to be checked by a small trusted core.

In the following, we assume the definitions in standard library: https://coq.inria.fr/stdlib/Coq.Init.Datatypes.html.

**Tactics 101**. Let's see how to use the common tactics to do proofs in Coq. We first define the theorem to be proved:


In [ ]:
Theorem neg_fun: forall P: Prop, P -> ~~P.

From the output, we see there is one proof goal. We start proof by writing `Proof`

In [ ]:
Proof.

If the goal is universal quantification or implication, we can use `intros` to introduce the conditions as premises:

In [ ]:
intros.

As introduced before, `~P` is defined as `P -> False`. We can use `unfold` to expand the definition:

In [ ]:
unfold not.

Now we can use `intros` again:

In [ ]:
intros.

Now we see that the proof goal is the return type of `H0`, the tatic `apply` is useful in this case:

In [ ]:
apply H0.

Now we see the proof goal `P` is already a given premise, we can use `exact` to complete the proof:

In [ ]:
exact H.

Now we can end the proof by `QED`:

In [ ]:
Qed.

We put all the proof steps together for readability:

```Coq
Theorem neg_fun: forall P: Prop, P -> ~~P.
Proof.
  intros. unfold not. intros. apply H0. exact H.
Qed.
```

**Proof by simplification**. When the proof goals involves definition of functions, the tatic `simpl` is handy. For example, we may prove

In [ ]:
Theorem plus_11: 7 + 4 = 6 + 5.
Proof.

In [ ]:
simpl.

The tatic `reflexivity` can be use to prove `a = a` and completes the proof: 

In [ ]:
reflexivity.
Qed.

**Proof by rewriting**. To take advantage of known equalities, we can use the tactic `rewrite`. For example, we may prove

In [ ]:
Theorem eq_rewrite: forall m n:nat, m = n -> m + 2 = n + 2.
Proof.
  intros m n H. rewrite -> H. reflexivity.
Qed.

**Proof by case analysis**. In the following, we can use `destruct` on do case analysis on the value of `b`:

In [ ]:
Theorem neg_eql : forall b : bool, negb (negb b) = b.
Proof.
  intros b. destruct b.
  (* case 1 *) simpl. reflexivity.
  (* case 2 *) simpl. reflexivity.
Qed.


**Proof by induction**. Here we can redo the proof of `even(double(n))` with the tactic `induction`:

In [ ]:
Fixpoint double (n : nat) : nat :=
  match n with
    | 0 => 0
    | S n' => S (S (double n'))
  end.

Inductive even : nat -> Prop :=
  | even0 : even 0
  | evenS : forall n:nat, even n -> even (S (S n)).

Theorem even_prop: forall n:nat, even (double n).
Proof.
  intros n. induction n as [|n' IH].
  (* case 1 *) simpl. constructor.
  (* case 1 *) simpl. constructor. exact IH.
Qed.

You can find the full list of tactics in Coq [here](https://coq.inria.fr/refman/coq-tacindex.html).

**Exercise 3**: Prove commutativity of addition (tip: prove a lemma first):

In [ ]:
Theorem add_commutativity: forall m n:nat, m + n = n + m.
Proof. Admitted.

## The equivalence between LEM and DNE

Curry-Howard correspondence is usually formulated only for _intuitionistic logics_ (IL), in which the _law of excluded middle_ (LEM) or equivalently the _law of double negation_ (DNE) does not hold. Concretely, the following propositions
are not provable in IL, thus by the correspondence there exists no programs in Coq that prove them:

- LEM: $\forall P.P \vee \neg P$
- DNE: $\forall P.\neg \neg P \to P$

This problem is about proving that intuitionistic logic with the law of excluded
middle is equivalent to intuitionistic logic with the law of double negation,
that is IL + LEM = IL + DNE.

First, we formulate the proof goal formally:

In [ ]:
Definition LEM_DNE_EQ: Prop := (forall P: Prop, P \/~P) <-> (forall P: Prop, ~~P -> P).

The forward direction is relatively easy:

In [ ]:
Theorem LEM_DNE: (forall P: Prop, P \/~P) -> (forall P: Prop, ~~P -> P).
Proof.
  intros H P nnp. destruct (H P) as [p|np].
  exact p. destruct (nnp np).
Qed.

The reverse direction needs some trick, in particular the three usage of `pose`:

In [ ]:
Theorem DNE_LEM: (forall P: Prop, ~~P -> P) -> (forall P: Prop, P \/~P).
Proof.
  intros H P. pose (H (P \/ ~P)) as H1.
  apply H1. intros H2.
  pose (fun p:P => H2 (or_introl p)) as H3.
  pose (fun p:~P => H2 (or_intror p)) as H4.
  apply H4.
  exact H3.
Qed.

Now we can just combine the two sub-proofs together:

In [ ]:
Definition proof_for_eq := conj LEM_DNE DNE_LEM.

Let's check if the proof indeed has the right type:

In [ ]:
Check proof_for_eq : LEM_DNE_EQ.

Voila! To understand better how the proof works, the following exercise is highly recommended:

**Exercise 4**: Redo the proof by implementing a function that has the type `LEM_DNE_EQ` without using tactics.

## Going further

We only scratched the surface of Coq. The following resources are useful if you want to go further:

- [Software Foundations](https://softwarefoundations.cis.upenn.edu/current/index.html)
- [Type Safety in Three Easy Lemmas](http://siek.blogspot.com/2013/05/type-safety-in-three-easy-lemmas.html) and the [code](https://gist.github.com/dkrustev/5890291)
- [The calculus of constructions](https://hal.inria.fr/file/index/docid/76024/filename/RR-0530.pdf), _T. Coquand, Gérard Huet_, 1986
- [Calculus of Inductive Constructions](https://coq.inria.fr/distrib/current/refman/language/cic.html)